メモ
I91,95,84あたりは多い

<table>
  <thead>
  <tr>
    <td>更新日時</td>
    <td>変更内容</td>    
  </tr>
  </thead>
  <tr>
    <td>0611 1230</td>
    <td>locationあり、officerなし。良さげ。</td> 
  </tr>
  <tr>
    <td>0613 1000</td>
    <td>dayを追加。location/officerは追加</td> 
  </tr>
  <tr>
    <td>0613 1430</td>
    <td>dayを追加。location/officerは追加</td> 
  </tr>
  <tr>
    <td>0613 1730</td>
    <td>locationのうち小さいものを除去するなど。officerなし。良さげ。</td> 
  </tr>
  <tr>
    <td>0613 1800</td>
    <td>location(>50)、officer(>1000)を追加。安定して0.88でる。</td> 
  </tr>
</table>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [85]:
df = pd.read_csv("test.csv")
ctlist = pd.read_csv("CTlist.csv")

In [86]:
df.head(3)

,state,stop_date,stop_time,location_raw,county_name,county_fips,fine_grained_location,police_department,driver_gender,driver_age_raw,...,driver_race_raw,driver_race,violation_raw,violation,search_conducted,search_type_raw,search_type,contraband_found,officer_id,stop_duration
0,CT,2014-10-13,07:18,orange,New Haven County,9009.0,"00000 S RT 15 (ORANGE, T107)",State Police,M,42,...,Black,Black,Moving Violation,Moving violation,False,NaN,NaN,False,19172,1-15 min
1,CT,2014-11-25,03:25,woodbury,Litchfield County,9005.0,RT 6,State Police,F,59,...,White,White,Registration,Registration/plates,False,NaN,NaN,False,1000002390,1-15 min
2,CT,2015-03-23,09:32,norwich,New London County,9011.0,i395nb x82,State Police,M,42,...,White,White,Other/Error,Other,False,NaN,NaN,False,1000003180,30+ min


In [87]:
ctlist.head(3)

,Unnamed: 0,Town,Designation,Date,Land area,Population,Form of,County,Council of Governments,Native American,X,Y
0,0,andover,Town,1848,15.46,"3,303",Town meeting,Tolland County,Capitol Region,NaN,41.737321,-72.370360
1,1,ansonia,City,1889,6.03,"19,249",Mayor-council,New Haven County,Naugatuck Valley,NaN,41.346208,-73.078997
2,2,ashford,Town,1714,38.79,"4,100",Town meeting,Windham County,Northeast CT,NaN,41.873066,-72.121616


以下、データの成形を行なっていく。

Violationに関しては、カンマ区切りにされているviolationの種類をstr.splitで分割した上で、全てのviolationの種類をまとめたリストを作成する。このリストをカラム名とし、各indexについて対応するviolationの値を1とする。

In [88]:
# violationの種類を検索
df_violation = df["violation"].str.split(',',expand=True)
violation_list = np.array([])

"""
for i in range(0,4):
    violation_list = np.append(violation_list,df_violation[0].unique())
violation_list = np.unique(violation_list)
"""

violation_list = np.array(['Cell phone', 'Equipment', 'License', 'Lights', 'Moving violation',
       'Other', 'Registration/plates', 'Safe movement', 'Seat belt',
       'Speeding', 'Stop sign/light'], dtype=object)

for list in violation_list:
    df["violation_"+list] = np.zeros(df.contraband_found.count())
    for i in range(0,4):
        df.loc[df_violation[i]==list,"violation_"+list]=1

In [89]:
# stop_durationを数値に直す
# 2回目以降の実行ではエラー
df.loc[df["stop_duration"]=="1-15 min","stop_duration"]=0
df.loc[df["stop_duration"]=="16-30 min","stop_duration"]=1
df.loc[df["stop_duration"]=="30+ min","stop_duration"]=2

# driver_genderを数値に直す
# 2回目以降の実行ではエラー
df.loc[df["driver_gender"]=="M","driver_gender"]=0
df.loc[df["driver_gender"]=="F","driver_gender"]=1

In [90]:
df['stop_time'] = pd.to_datetime(df['stop_time'])
df['stop_date'] = pd.to_datetime(df['stop_date'])
df["stop_weekday_name"] = df["stop_date"].dt.weekday_name
df["stop_month"] = df["stop_date"].dt.month    
df["stop_year"] = df["stop_date"].dt.year
df["stop_hour"] = df["stop_time"].dt.hour
df["stop_day"] = df["stop_date"].dt.day

In [91]:
df["location_raw"] = df["location_raw"].fillna("N/A")
df["fine_grained_location"] = df["fine_grained_location"].fillna("N/A")

In [92]:
df["location_exit"] = 0
df.loc[df["fine_grained_location"].str.contains("X"),"location_exit"] = 1
df.loc[df["fine_grained_location"].str.contains("x"),"location_exit"] = 1
df.loc[df["fine_grained_location"].str.contains("sex"),"location_exit"] = 0

df["location_route"] = 0
df.loc[df["fine_grained_location"].str.contains("rt"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("rte "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("RT."),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("RT "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("rT "),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Route"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Rte"),"location_route"] = 1
df.loc[df["fine_grained_location"].str.contains("Rt"),"location_route"] = 1

df["location_00000"] = 0
df.loc[df["fine_grained_location"].str.contains("000"),"location_00000"] = 1

In [93]:
df.loc[df["location_raw"]=="e lyme","location_raw"] = "east lyme"
df.loc[df["location_raw"]=="e hartford","location_raw"] = "east hartford"
df.loc[df["location_raw"]=="e granby","location_raw"] = "east granby"
df.loc[df["location_raw"]=="no stonington","location_raw"] = "north stonington"
df.loc[df["location_raw"]=="no canaan","location_raw"] = "north canaan"
df.loc[df["location_raw"]=="no branford","location_raw"] = "north branford"
df.loc[df["location_raw"]=="falls village","location_raw"] = "canaan" #CDP
df.loc[df["location_raw"]=="ivoryton","location_raw"] = "essex" #CDP
df.loc[df["location_raw"]=="sprague - baltic","location_raw"] = "sprague" #CDP 
df.loc[df["location_raw"]=="sprague - versailles","location_raw"] = "sprague" #? #
df.loc[df["location_raw"]=="storrs","location_raw"] = "mansfield" # CDP
df.loc[df["location_raw"]=="centerbrook","location_raw"] = "essex" #hamletって場所らしい
df.loc[df["location_raw"]=="griwold","location_raw"] = "griswold"
df.loc[df["location_raw"]=="uncasville","location_raw"] = "montville"
df.loc[df["location_raw"]=="winsted","location_raw"] = "litchfield"
df.loc[df["location_raw"]=="washighton","location_raw"] = "washington"
df.loc[df["location_raw"]=="danielson","location_raw"] = "killingly"
df.loc[df["location_raw"]=="t165","location_raw"] = "windsor locks"

df.loc[(df["county_fips"].isnull()) & (df["fine_grained_location"].str.contains("T045")),["location_raw","county_name","county_fips"]] = ["east lyme","New London County","9011.0"]
df.loc[(df["county_fips"].isnull()) & (df["fine_grained_location"].str.contains("t045")),["location_raw","county_name","county_fips"]] = ["east lyme","New London County","9011.0"]

In [94]:
df = pd.get_dummies(df,columns=["stop_month"], prefix='stop_month', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_year"], prefix='stop_year', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_weekday_name"], prefix='stop_day', prefix_sep='_')
df = pd.get_dummies(df,columns=["stop_hour"], prefix='stop_hour', prefix_sep='_')

df = pd.get_dummies(df,columns=["driver_race"],prefix='race',prefix_sep='_')

df = pd.get_dummies(df,columns=["search_type"], prefix='search', prefix_sep='_')

df = pd.get_dummies(df,columns=["county_name"], drop_first=True, prefix='county', prefix_sep='_')

In [95]:
df = df.drop(["stop_date","stop_time","search_type_raw","search_conducted","county_fips"
              ,"driver_race_raw","violation","violation_raw","state"
              ,"police_department","driver_age"],axis=1)

In [96]:
df["contraband_found"] = df["contraband_found"]*1

In [97]:
ctlist["Population"] = ctlist["Population"].str.replace(',','').astype(np.int)

In [98]:
df.loc[df["location_raw"].isnull(),"location_raw"]="N/A"

In [99]:
population_dict = dict(zip(ctlist['Town'], ctlist['Population']))
area_dict = dict(zip(ctlist['Town'], ctlist['Land area']))
X_dict = dict(zip(ctlist['Town'], ctlist['X']))
Y_dict = dict(zip(ctlist['Town'], ctlist['Y']))
population_dict.update({"N/A":ctlist["Population"].mean()})
area_dict.update({"N/A":ctlist["Land area"].mean()})
X_dict.update({"N/A":ctlist["X"].mean()})
Y_dict.update({"N/A":ctlist["Y"].mean()})

In [100]:
df["population"] = df["location_raw"].replace(population_dict).astype(np.int)
df["area"] = df["location_raw"].replace(area_dict).astype(np.float)
df["X"] = df["location_raw"].replace(X_dict).astype(np.float)
df["Y"] = df["location_raw"].replace(Y_dict).astype(np.float)

In [101]:
df["location_raw"].unique()
location_list = np.array(['columbia', 'sherman', 'colchester', 'west haven', 'haddam',
       'mansfield', 'stonington', 'marlborough', 'griswold', 'milford',
       'norwalk', 'bridgewater', 'tolland', 'deep river', 'union',
       'roxbury', 'cornwall', 'woodbridge', 'shelton', 'brooklyn',
       'danbury', 'wallingford', 'groton', 'litchfield', 'oxford',
       'naugatuck', 'new london', 'brookfield', 'plainfield', 'scotland',
       'wethersfield', 'north haven', 'middletown', 'north canaan',
       'trumbull', 'bethel', 'westbrook', 'killingly', 'lisbon',
       'bridgeport', 'somers', 'old saybrook', 'east granby', 'vernon',
       'winchester', 'burlington', 'east hartford', 'ellington', 'kent',
       'new milford', 'hartford', 'morris', 'willington', 'middlebury',
       'north stonington', 'chester', 'middlefield', 'stafford',
       'westport', 'old lyme', 'thompson', 'goshen', 'manchester',
       'windsor', 'stamford', 'farmington', 'essex', 'east haddam',
       'stratford', 'colebrook', 'waterbury', 'greenwich', 'harwinton',
       'guilford', 'beacon falls', 'madison', 'watertown', 'waterford',
       'lebanon', 'windsor locks', 'new canaan', 'franklin', 'salisbury',
       'fairfield', 'preston', 'new haven', 'putnam', 'andover',
       'woodbury', 'salem', 'ashford', 'enfield', 'clinton', 'cromwell',
       'bolton', 'southbury', 'newtown', 'durham', 'ansonia', 'montville',
       'washington', 'branford', 'east lyme', 'norwich', 'new hartford',
       'thomaston', 'hamden', 'canaan', 'darien', 'hebron', 'pomfret',
       'voluntown', 'derby', 'ledyard', 'new fairfield', 'eastford',
       'new britain', 'meriden', 'west hartford', 'bristol', 'windham',
       'berlin', 'rocky hill', 'monroe', 'glastonbury', 'killingworth',
       'orange', 'torrington', 'east windsor', 'southington', 'bozrah',
       'seymour', 'bethany', 'barkhamsted', 'cheshire', 'east haven',
       'norfolk', 'east hampton', 'sharon', 'sterling', 'north branford',
       'plainville', 'hampton', 'avon', 'sprague', 'woodstock', 'chaplin',
       'coventry', 'prospect', 'south windsor', 'canton', 'canterbury',
       'newington', 'bethlehem', 'portland', 'hartland', 'warren', 'lyme',
       'easton', 'plymouth', 'bloomfield', 'ridgefield', 'suffield',
       'wolcott', 'simsbury', 'granby','wilton','orange'],
      dtype=object)

In [102]:
df["officer_id"].unique()[df["officer_id"].value_counts()>100]

array([     19172, 1000002390, 1000003180,  756261343,  557714559,
       1000002845, 1000002575, 1000003203, 1000003222, 1000002570,
        679323750, 1000002265, 1000002712,   48889962, 1000002955,
        790642042, 1000001899,   25413887, 1000002266, 1000002938,
       1000002710, 1000002760, 1000003192, 1000002369, 1000003034,
       1000003234,  827637066,   71080948,   38634270,  956596331,
         39333217, 1000003162, 1000002578,  432675854, 1000002705,
       1000002240, 1000002961, 1000002234,  120121335,  117829510,
       1000001884,  498167384, 1000002271, 1000002370, 1000002256,
       1000002608,  381521852, 1000002362, 1000001948, 1000002706,
        759276770,  671622127, 1000002723, 1000003375, 1000003182,
       1000003154,      19018, 1000002562,    8861580, 1000003176,
       1000001898, 1000002371, 1000002702,  816078115, 1000003231,
       1000003198, 1000001940,  335699444,  452365504,   88185785,
       1000002715, 1000003196, 1000003364, 1000002735, 1000003

In [103]:
df["officer_id"].unique()[df["officer_id"].value_counts()>50]
officer_list = np.array([55887736, 770626977, 1000002961, 1000002376, 70583331, 1000002714,
       1000003026, 1000002219, 298463910, 1000002220, 1000003191,
       1000003177, 1000002710, 1000003167, 653242422, 1000001884,
       277048371, 1000003181, 1000003235, 1000001983, 1000002599,
       1000003171, 1000003221, 1000002238, 1000002587, 1000002944,
       1000003198, 742774399, 1000002946, 540042328, 614680420,
       1000002270, 483102647, 104759822, 159668340, 1000001956, 642677890,
       1000002353, 469479530, 1000002703, 1000002613, 934977327,
       1000003163, 1000002591, 747377440, 1000002244, 1000002259,
       953549550, 1000002256, 853847642, 702562013, 1000002569,
       1000002708, 710252242, 116991428, 1000001923, 1000003230,
       322409049, 1000002750, 308565279, 1000003205, 637071474,
       1000002702, 1000002959, 1000002208, 1000002479, 1000003211,
       1000001939, 19018, 706750556, 1000002747, 1000003213, 520878322,
       1000002608, 1000003215, 1000002385, 927601465, 31240166, 331752178,
       1000003196, 1000002377, 1000002761, 18907, 1000001893, 1000003021,
       607163196, 1000002546, 257011101, 829157681, 26013717, 1000002235,
       230931989, 668908943, 722065405, 1000003194, 533883416, 944661814,
       915131263, 1000002728, 824655496, 908158929, 1000002362,
       1000003174, 1000002748, 599200807, 1000002240, 1000003017,
       1000002727, 1000002875, 1000003162, 1000003229, 1000001940,
       1000003363, 726828835, 872476022, 1000001996, 190163716,
       1000003156, 1000002617, 1000002598, 1000002940, 1000002575,
       1000003182, 265069536, 71903124, 1000002939, 1000003387,
       1000002349, 960967070, 424481709, 1000003031, 1000002952,
       713140385, 1000002380, 413162233, 668836964, 1000003186, 124039771,
       1000002576, 424133385, 1000002760, 1000003203, 13468937, 329593147,
       1000003055, 1000003029, 293708152, 567049933, 1000003160, 78218753,
       785469339, 1000002237, 41354688, 1000003384, 1000003022,
       1000003272, 940662250, 335699444, 279965403, 1000003353, 725428335,
       807433607, 1000002740, 418117108, 1000001914, 1000001949,
       1000003195, 826689753, 1000002726, 1000002602, 1000003223,
       684190569, 1000002954, 106840086, 1000003225, 1000002634,
       439075424, 1000002623, 1000002243, 19172, 1000002960, 1000003392,
       1000002371, 1000002271, 1000002097, 1000002721, 381521852,
       1000002942, 418720717, 1000003168, 253215760, 89095882, 1000003399,
       1000003208, 29992679, 498167384, 495322141, 861164535, 1000002949,
       1000002217, 1000002590, 1000003034, 1000002758, 1000001920,
       1000003175, 1000003155, 1000003189, 1000002743, 1000003207,
       1000002957, 1000001918, 790642042, 506911367, 18893, 961269420,
       1000003197, 1000002955, 1000001903, 983655904, 899734338, 46852777,
       565408419, 1000003154, 415391716, 1000002732, 1000003092,
       1000001917, 562015716, 1000002762, 708150264, 1000003037,
       1000003377, 1000002733, 19012, 577714579, 324857301, 671622127,
       488417172, 167438590, 1000002730, 1000002763, 1000002265,
       697576848, 19209, 1000001922, 106659415, 1000002855, 1000002752,
       1000003176, 941674240, 939084757, 569478224, 88120, 512066154,
       1000002549, 1000003166, 1000002618, 363177935, 23041953,
       1000001885, 1000002604, 1000002363, 634601902, 1000002906,
       1000002606, 1000002707, 607702361, 480852965, 1000003310,
       113658284, 1000003124, 1000002578, 480991900, 201145055, 178825628,
       241315469, 1000001899, 11486393, 94016, 627195004, 381140169,
       1000002715, 1000002233, 726303957, 88185785, 1000003200, 875724425,
       1000002593, 1000002709, 1000002586, 448051592, 987312802,
       1000003234, 1000003192, 907060051, 462749467, 258421805,
       1000003164, 848339269, 641939147, 938887034, 1000002624,
       1000001952, 1000003032, 970826817, 1000001935, 1000002724,
       1000003397, 1000002207, 61208768, 1000002204, 1000003169,
       1000001927, 349680451, 1000003183, 1000002869, 120121335,
       556481407, 336159473, 1000002384, 257227481, 759517227, 1000001941,
       612222124, 281119621, 923295155, 1000002723, 830814942, 1000003227,
       910711431, 1000003218, 639567735, 389731444, 71080948, 1000003024,
       764772746, 1000002221, 1000001998, 735104535, 1000002565,
       1000002592, 1000002741, 288030420, 95565661, 1000003352,
       1000002950, 1000002369, 849715205, 1000003356, 1000001948,
       1000002625, 23609079, 1000003368, 240586212, 445842676, 10450,
       351062524, 447203558, 1000002895, 1000002705, 827637066,
       1000002600, 18936, 195480226, 680984616, 1000002370, 1000003226,
       1000002352, 347486231, 1000002711, 1000002269, 592196034,
       1000002755, 857234458, 1000002953, 487371315, 223693286, 803023284,
       1000002620, 1000002725, 1000002298, 221680160, 349527, 596608526,
       1000003170, 1000002641, 1000003187, 759276770, 39333217,
       1000002390, 204876707, 1000002746, 536623146, 1000002734,
       172424611, 299543405, 1000002254, 175841017, 618526899, 100117168,
       1000002562, 1000001891, 831694856, 1000003188, 1000002742,
       1000002584, 1000003345, 1000002945, 703453720, 1000002234,
       1000002735, 19200, 849977595, 1000002564, 19208, 583155154,
       1000002477, 1000003228, 1000002245, 1000003210, 1000003372,
       1000003358, 1000002894, 1000003366, 1000003180, 1000002574,
       669318187, 8715299, 18877, 18889, 1000003350, 1000002350,
       967818854, 1000002098, 1000002232, 540948842, 1000002850,
       1000002713, 16934571, 1000002372, 1000002718, 1000002605,
       1000003179, 720397369, 1000001913, 1000002249, 1000002645,
       1000002947, 455920825, 1000001890, 277059361, 812220713,
       1000002266, 85290706, 705312695, 1000003036, 1000002951, 951755976,
       1000002745, 1000002367, 1000002251, 1000003172, 965123888,
       1000003185, 1000002622, 1000003365, 1000003199, 868524004,
       1000003161, 1000003157, 435922450, 337829011, 1000001911,
       140170252, 684674308, 1000001933, 1000003224, 708882849,
       1000001944, 1000003065, 184395432, 1000002198, 593650992,
       1000003360, 1000003173, 1000003393, 1000003222, 1000002690,
       967339756, 511727371, 432175311, 1000002366, 840996875, 1000003178,
       1000003379, 1000002381, 813172081, 533260202, 512748515, 356518293,
       38634270, 959449555, 1000002570, 167327553, 1000001895, 225533692,
       892380506, 519033587, 757078334, 1000002336, 958336317, 539411814,
       160269181, 225058271, 1000003132, 136981404, 269460408, 1000002365,
       123065616, 1000003142, 463913800, 18922, 1000002246, 811927512,
       909702662, 449139270, 717674300, 1000001916, 1000003209, 747982770,
       117829510, 1000002938, 775049359, 1000002559, 444277014,
       1000003184, 1000002736, 1000002357, 1000003202, 1000003265, 19211,
       1000001898, 18960, 1000001930, 1000002595, 201646350, 1000002846,
       1000002250, 890765642, 1000002722, 413508965, 1000002573,
       1000002756, 1000002948, 1000003233, 1000001880, 1000002685,
       585195195, 911957941, 1000003193, 1000003060, 896134234,
       1000003018, 1000002616, 1000002585, 616978485, 1000003357,
       1000002956, 1000003165, 1000002545, 1000003204, 1000003212,
       1000002597, 738493170, 1000002364, 412419749, 452365504,
       1000003344, 1000002969, 180287375, 403738932, 1000002230,
       585628821, 1000001947, 25413887, 679323750, 1000002211, 8861580,
       263341120, 1000002589, 1000003389, 940080406, 779497884,
       1000003030, 890059831, 1000003027, 1000003158, 1000003419,
       1000002388, 1000002885, 1000001954, 1000002716, 763593461,
       48889962],
      dtype=object)

In [104]:
%%time
for list in location_list:
    df["location_"+list] = np.zeros(df.contraband_found.count())
    df.loc[df["location_raw"]==list,"location_"+list]=1

CPU times: user 23 s, sys: 11.7 s, total: 34.7 s
Wall time: 17.7 s


In [105]:
%%time
for list in officer_list:
    df["officer_"+str(list)] = np.zeros(df.contraband_found.count())
    df.loc[df["location_raw"]==str(list),"officer_"+str(list)]=1

CPU times: user 4min 25s, sys: 2min 56s, total: 7min 21s
Wall time: 4min 25s


In [106]:
df["location_I84"] = 0
df.loc[(df["fine_grained_location"].str.contains("84")),"location_I84"] = 1

df["location_I91"] = 0
df.loc[(df["fine_grained_location"].str.contains("91")) & ~(df["fine_grained_location"].str.contains("T091")),"location_I91"] = 1

df["location_I95"] = 0
df.loc[(df["fine_grained_location"].str.contains("95")) & ~(df["fine_grained_location"].str.contains("T095")),"location_I95"] = 1

df["location_I395"] = 0
df.loc[(df["fine_grained_location"].str.contains("395")) & ~(df["fine_grained_location"].str.contains("T395")),"location_I395"] = 1

In [107]:
df = df.drop(["fine_grained_location","location_raw","officer_id"],axis=1)

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93994 entries, 0 to 93993
Columns: 859 entries, driver_gender to location_I395
dtypes: float64(785), int64(13), uint8(61)
memory usage: 577.7 MB


In [109]:
df.to_csv('test_clean_06181111.csv',sep=',',index=False)